In [ ]:
!pip install featuretools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.9/587.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.2/215.2 kB 10.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
customers_df = pd.read_csv("/content/drive/MyDrive/Data Mining/customersdf.csv")
orders_df = pd.read_csv("/content/drive/MyDrive/Data Mining/ordersdf.csv")
products_df = pd.read_csv("/content/drive/MyDrive/Data Mining/productsdf.csv")
order_details_df = pd.read_csv("/content/drive/MyDrive/Data Mining/orderdetailsdf.csv")

In [ ]:
customers_df.head(5)

,Customer ID,Customer Name,Segment,City,State,Email,Sign Up Date
0,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,claire@example.com,11/08/2014
1,DV-13045,Darrin Van Huff,Corporate,Los Angeles,California,darrin@example.com,11/12/2014
2,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,sean@example.com,07/05/2014
3,BH-11710,Brosina Hoffman,Consumer,Los Angeles,California,brosina@example.com,24/05/2014
4,AA-10480,Andrew Allen,Consumer,Concord,North Carolina,andrew@example.com,26/08/2014


In [ ]:
orders_df.head(5)

,Order ID,Order Date,Ship Date
0,CA-2016-152156,08/11/2016,11/11/2016
1,CA-2016-138688,12/06/2016,16/06/2016
2,US-2015-108966,11/10/2015,18/10/2015
3,CA-2014-115812,09/06/2014,14/06/2014
4,CA-2017-114412,15/04/2017,20/04/2017


In [ ]:
products_df.head(5)

,Product ID,Product Name,Category,Sub-Category
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture,Bookcases
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels
3,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables
4,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Office Supplies,Storage


In [ ]:
order_details_df.head(5)

,Order ID,Customer ID,Product ID,Quantity,Discount,Sales,Profit,Order Detail ID
0,CA-2016-152156,CG-12520,FUR-BO-10001798,2,0.00,261.9600,41.9136,0
1,CA-2016-152156,CG-12520,FUR-CH-10000454,3,0.00,731.9400,219.5820,1
2,CA-2016-138688,DV-13045,OFF-LA-10000240,2,0.00,14.6200,6.8714,2
3,US-2015-108966,SO-20335,FUR-TA-10000577,5,0.45,957.5775,-383.0310,3
4,US-2015-108966,SO-20335,OFF-ST-10000760,2,0.20,22.3680,2.5164,4


In [ ]:
import featuretools as ft

In [ ]:
customers_df['Sign Up Date'] = pd.to_datetime(customers_df['Sign Up Date'], format='%d/%m%Y', errors='coerce')
orders_df['Order Date'] = pd.to_datetime(orders_df['Order Date'], format='%d/%m%Y', errors='coerce')
orders_df['Ship Date'] = pd.to_datetime(orders_df['Ship Date'], format='%d/%m%Y', errors='coerce')

In [ ]:
# Create an EntitySet
es = ft.EntitySet(id="ecommerce")

# Add entities to the EntitySet
es = es.add_dataframe(dataframe_name="customers", dataframe=customers_df, index="Customer ID")
es = es.add_dataframe(dataframe_name="orders", dataframe=orders_df, index="Order ID")
es = es.add_dataframe(dataframe_name="products", dataframe=products_df, index="Product ID")
es = es.add_dataframe(dataframe_name="order_details", dataframe=order_details_df, index="Order Detail ID")

/usr/local/lib/python3.10/dist-packages/featuretools/entityset/entityset.py:724: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index
  warnings.warn(


In [ ]:
relationships = [
    ("customers", "Customer ID", "order_details", "Customer ID"),
    ("orders", "Order ID", "order_details", "Order ID"),
    ("products", "Product ID", "order_details", "Product ID"),
]

es.add_relationships(relationships)

Entityset: ecommerce
  DataFrames:
    customers [Rows: 793, Columns: 7]
    orders [Rows: 5009, Columns: 3]
    products [Rows: 1862, Columns: 4]
    order_details [Rows: 9993, Columns: 8]
  Relationships:
    order_details.Customer ID -> customers.Customer ID
    order_details.Order ID -> orders.Order ID
    order_details.Product ID -> products.Product ID

In [ ]:
# Perform feature engineering
feature_matrix, feature_defs = ft.dfs(entityset=es, target_dataframe_name="order_details")

/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function min at 0x7ebafd7f8dc0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function max at 0x7ebafd7f8ca0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x7ebafd7f8670> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will b

In [ ]:
feature_matrix

,Quantity,Discount,Sales,Profit,customers.Segment,customers.State,products.Category,products.Sub-Category,customers.COUNT(order_details),customers.MAX(order_details.Discount),...,products.STD(order_details.Discount),products.STD(order_details.Profit),products.STD(order_details.Quantity),products.STD(order_details.Sales),products.SUM(order_details.Discount),products.SUM(order_details.Profit),products.SUM(order_details.Quantity),products.SUM(order_details.Sales),products.NUM_CHARACTERS(Product Name),products.NUM_WORDS(Product Name)
Order Detail ID,,,,,,,,,,,,,,,,,,,,,
0,2,0.00,261.9600,41.9136,Consumer,Kentucky,Furniture,Bookcases,5,0.60,...,0.209662,55.336789,1.414214,178.021609,0.85,-56.3214,12.0,1263.957,33,4
1,3,0.00,731.9400,219.5820,Consumer,Kentucky,Furniture,Chairs,5,0.60,...,0.124011,170.137853,2.454125,508.523764,1.70,1927.4420,51.0,10637.528,59,8
2,2,0.00,14.6200,6.8714,Corporate,California,Office Supplies,Labels,9,0.45,...,0.106904,8.338894,2.429972,17.482977,0.60,64.4011,23.0,153.510,57,7
3,5,0.45,957.5775,-383.0310,Consumer,Florida,Furniture,Tables,15,0.45,...,0.182737,211.860513,1.505941,325.454627,2.10,-532.7613,29.0,7242.768,45,6
4,2,0.20,22.3680,2.5164,Consumer,Florida,Office Supplies,Storage,15,0.45,...,0.109545,17.874292,3.898718,55.590499,0.60,63.7488,24.0,301.968,30,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,3,0.20,25.2480,4.1028,Consumer,Washington,Furniture,Furnishings,17,0.20,...,0.103280,9.522024,2.509980,26.177684,0.40,74.7972,27.0,265.104,22,4
9989,2,0.00,91.9600,15.6332,Consumer,Washington,Furniture,Furnishings,13,0.60,...,0.075593,17.425377,2.058663,84.258839,0.20,124.6058,23.0,1002.364,50,9
9990,2,0.20,258.5760,19.3932,Consumer,Washington,Technology,Phones,13,0.60,...,0.151186,142.667335,2.811541,457.754645,1.00,575.3316,26.0,3684.708,21,4


In [ ]:
feature_defs

[<Feature: Quantity>,
 <Feature: Discount>,
 <Feature: Sales>,
 <Feature: Profit>,
 <Feature: customers.Segment>,
 <Feature: customers.State>,
 <Feature: products.Category>,
 <Feature: products.Sub-Category>,
 <Feature: customers.COUNT(order_details)>,
 <Feature: customers.MAX(order_details.Discount)>,
 <Feature: customers.MAX(order_details.Profit)>,
 <Feature: customers.MAX(order_details.Quantity)>,
 <Feature: customers.MAX(order_details.Sales)>,
 <Feature: customers.MEAN(order_details.Discount)>,
 <Feature: customers.MEAN(order_details.Profit)>,
 <Feature: customers.MEAN(order_details.Quantity)>,
 <Feature: customers.MEAN(order_details.Sales)>,
 <Feature: customers.MIN(order_details.Discount)>,
 <Feature: customers.MIN(order_details.Profit)>,
 <Feature: customers.MIN(order_details.Quantity)>,
 <Feature: customers.MIN(order_details.Sales)>,
 <Feature: customers.SKEW(order_details.Discount)>,
 <Feature: customers.SKEW(order_details.Profit)>,
 <Feature: customers.SKEW(order_details.Quan